In [8]:
from enterprise_extensions.models import model_general, model_2a
from enterprise.pulsar import Pulsar

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
psrlist = None # define a list of pulsar name strings that can be used to filter.
# set the data directory
datadir = './data'
if not os.path.isdir(datadir):
    datadir = '../../data'
print(datadir)

./data


In [6]:
# get only J1909
psrname = 'J1909'
parfiles = sorted(glob.glob(datadir + '/par/' + psrname + '*par'))
timfiles = sorted(glob.glob(datadir + '/tim/' + psrname + '*tim'))

print(parfiles)
print(timfiles)

['./data/par/J1909-3744_NANOGrav_12yv3.gls.par']
['./data/tim/J1909-3744_NANOGrav_12yv3.tim']


In [7]:
psrs = []
ephemeris = 'DE438'
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem=ephemeris)
    psrs.append(psr)

In [9]:
## Get parameter noise dictionary
noise_ng12 = datadir + '/channelized_12p5yr_v3_full_noisedict.json'

params = {}
with open(noise_ng12, 'r') as fp:
    params.update(json.load(fp))

In [33]:
pta = model_2a([psrs[0]], noisedict=params, gamma_common=13/3, n_gwbfreqs=5)

In [34]:
pta.get_chi([0, -20, -18])

[KernelMatrix([2.58419900e-15, 2.58419900e-15, 8.81203405e-17,
               8.81203405e-17, 1.24177119e-17, 1.24177119e-17,
               3.11906735e-18, 3.11906735e-18, 1.07394272e-18,
               1.07394272e-18, 2.80738403e-19, 2.80738403e-19,
               1.29887655e-19, 1.29887655e-19, 6.66205389e-20,
               6.66205389e-20, 3.69696662e-20, 3.69696662e-20,
               2.18302182e-20, 2.18302182e-20, 1.35548480e-20,
               1.35548480e-20, 8.77307508e-21, 8.77307508e-21,
               5.87951246e-21, 5.87951246e-21, 4.05898922e-21,
               4.05898922e-21, 2.87476124e-21, 2.87476124e-21,
               2.08189184e-21, 2.08189184e-21, 1.53749414e-21,
               1.53749414e-21, 1.15530207e-21, 1.15530207e-21,
               8.81637535e-22, 8.81637535e-22, 6.82194318e-22,
               6.82194318e-22, 5.34517099e-22, 5.34517099e-22,
               4.23588999e-22, 4.23588999e-22, 3.39171144e-22,
               3.39171144e-22, 2.74158596e-22, 2.741585

In [38]:
pta2 = model_general([psrs[0]], tm_svd=True, tm_linear=True, noisedict=params,
                     common_components=30, gamma_common=13/3, red_components=5
                    )

In [37]:
%%timeit
pta.get_lnlikelihood([0, -20, -18])

198 µs ± 884 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [41]:
%%timeit
pta2.get_lnlikelihood([0, -20, -18])

200 µs ± 703 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [42]:
print(pta.get_lnlikelihood([0, -20, -18]))
print(pta2.get_lnlikelihood([0, -20, -18]))

303871.070452288
303871.07046380057
